In [5]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.llms import openai
from langchain.chains import LLMChain, LLMRouterChain, MultiPromptChain, HypotheticalDocumentEmbedder, RetrievalQA
import dotenv
from langchain_core.prompts import PromptTemplate
from typing import Optional
import json
import pandas as pd
from Text_preprocessing import Text_preprocessing
from langchain_community.document_loaders import DataFrameLoader
from typing import List, Dict, Any, Mapping
from langchain.globals import set_debug

from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

from langchain_community.vectorstores import chroma as Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.vectorstores import VectorStoreRetriever
from pydantic.v1 import Field
from langchain_core.documents import Document
from langchain_community.document_transformers import (
    LongContextReorder
)
from sentence_transformers import CrossEncoder
import llmModels 
import wandb
import pickle
import prompts
import groundTruths

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI


In [27]:
dotenv.load_dotenv()
## Llama 3 8B
chatModel_llama3_8B = llmModels.loadLlama3_8B()
##FSD_1777
dataPath = "/home/mbhatti/mnt/d/LLM-repo1/models/langchain_implementation/FSD1777_Oct23.json"
dateFrom = "2023-10-19T09:00:00+00:00" #2023-10-19T18:58:41Z for 200 tweets
dateTo = "2023-10-19T18:00:00+00:00"


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.40s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
"""Load relevant fields of flood tags api json response"""
def json_dataloader(dataPath = dataPath, dateFrom = dateFrom, dateTo = dateTo):
    # Load json and extract relevant records in pandas df
    with open(dataPath, 'r') as json_file:
        response_dict = json.load(json_file)

    # Convert to pandas df    
    pd.set_option('display.max_colwidth', None)
    df = pd.DataFrame(response_dict)
    df['date'] = pd.to_datetime(df['date'])
    df = df.drop(columns=['id','tag_class', 'source', 'lang', 'urls','locations'])

    #Get data between thresholds
    threshold_datetime_lower = pd.to_datetime(dateFrom)
    threshold_datetime_upper = pd.to_datetime(dateTo)
    df = df[df['date'] >= threshold_datetime_lower]
    df = df[df['date'] <= threshold_datetime_upper]

    #Remove duplicates
    df  = df.drop_duplicates(subset=["text"], keep=False)
    #Pre-process
    preprocess = Text_preprocessing(df)
    df = preprocess.preprocess()
    #Covert date to string
    df['date'] = df['date'].astype(str)
    return df

def bgeEmbeddings():
    model_name = "BAAI/bge-large-en-v1.5"
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
    model = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    return model

def data_embedding(data : list, eModel = "bge-large-en-v1.5", rType = "Query", metric = "L2"):
    """Vectorize the data using OpenAI embeddings and store in Chroma db"""
    if eModel != "bge-large-en-v1.5":
        embeddings = OpenAIEmbeddings()
    else:
        embeddings = bgeEmbeddings()
    
    # if (rType == "Hyde"):
    #     embeddings = multiGen_hydeEmbedder(embeddings)

    documents = []
    loader = DataFrameLoader(data, page_content_column="text")
    documents.extend(loader.load())

    #Change this -- removal of duplicates
    db = Chroma.Chroma.from_documents(documents,embeddings)
    # if db._client.list_collections() != None:
    #     for collection in db._client.list_collections():
    #         ids = collection.get()['ids']
    #         print('REMOVE %s document(s) from %s collection' % (str(len(ids)), collection.name))
    #         if len(ids): collection.delete(ids)

    # #Create a vector store
    # if metric == "cosine":
    #     db = Chroma.Chroma.from_documents(documents,embeddings, collection_metadata={"hnsw:space": "cosine"})
    # else:
    #     db = Chroma.Chroma.from_documents(documents,embeddings)
    print(len(db._collection.get()['ids']))
    return db

In [29]:
if vectorstore._client.list_collections() != None:
    for collection in vectorstore._client.list_collections():
        ids = collection.get()['ids']
        print('REMOVE %s document(s) from %s collection' % (str(len(ids)), collection.name))
        if len(ids): collection.delete(ids)

REMOVE 480 document(s) from langchain collection


In [30]:
#  LLM initialisation
model = chatModel_llama3_8B

# Load the data from source
data = json_dataloader()

# Convert to vector store
vectorstore = data_embedding(data, eModel = "bge-large-en-v1.5", rType = "Query")

question = "**FLOOD WARNING** at Inverurie" #**FLOOD WARNING** at Inverurie
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={'k': 4}), llm=llm, include_original=True
) #search_kwargs={'k': 40}

# retriever = vectorstore.as_retriever(search_kwargs={'k': 40})


default_prompt = PromptTemplate(template = prompts.prompt_template_llama3_loc, input_variables = ['question', 'context'])
default_chain = RetrievalQA.from_chain_type(llm = model,
                        chain_type='stuff',
                        retriever=retriever_from_llm,
                        chain_type_kwargs={"prompt": default_prompt},
                        return_source_documents=True
                        )

ans = default_chain.invoke(question)
print(ans['query'])
print(ans['result'])
print(ans['source_documents'])

555


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**FLOOD WARNING** at Inverurie


According to the context, there is a **FLOOD WARNING** at Inverurie.
[Document(page_content='**FLOOD WARNING** at Inverurie', metadata={'date': '2023-10-19 17:34:15+00:00'}), Document(page_content='Localised Flood Warning issued for Findhorn, Nairn, Moray and Speyside region', metadata={'date': '2023-10-19 12:47:34+00:00'}), Document(page_content='flooding in Forfar Angus getting pretty bad with red warning due to storm babet', metadata={'date': '2023-10-19 17:33:40+00:00'}), Document(page_content="The Red warning's been extended to: Aberdeenshire Angus Dundee Perth &amp; Kinross This means extremely dangerous travel conditions and floodwater could cause a danger to life. Please avoid travel in these areas. More safety advice here:", metadata={'date': '2023-10-19 13:23:57+00:00'}), Document(page_content='Poor Brechin, may be told to evacuate. Hope all are safe. How to help? we need a thoughtful evaluation of all flood risk across Scotland. We need devel